In [27]:
from backtesting import Backtest, Strategy
import pandas as pd
import numpy as np
import pandas_ta as ta
import yfinance as yf
from backtesting.lib import crossover,plot_heatmaps,resample_apply
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt

In [31]:
#collecting data from yfinance

data = yf.download(tickers = 'ITC.NS',period = '7d',interval= '1m')
data

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2024-04-02 09:15:00+05:30,428.700012,428.700012,426.700012,426.700012,426.700012,0
2024-04-02 09:16:00+05:30,426.850006,427.200012,426.450012,427.200012,427.200012,76478
2024-04-02 09:17:00+05:30,427.049988,427.700012,427.049988,427.649994,427.649994,34740
2024-04-02 09:18:00+05:30,427.649994,427.649994,427.049988,427.100006,427.100006,40024
2024-04-02 09:19:00+05:30,427.049988,427.250000,426.100006,426.200012,426.200012,102324
...,...,...,...,...,...,...
2024-04-10 15:25:00+05:30,436.450012,436.450012,436.200012,436.399994,436.399994,60780
2024-04-10 15:26:00+05:30,436.399994,436.399994,436.100006,436.399994,436.399994,87429
2024-04-10 15:27:00+05:30,436.350006,436.399994,436.100006,436.399994,436.399994,90565


In [91]:
# Creating Strategy

class strategy1(Strategy):

    upper_bound_minute = 60
    lower_bound_minute = 45
    rsi_length_minute = 14
    upper_bound_2 = 83
    lower_bound_2 = 43
    rsi_length_2 = 14
    re = '5T'
    
    



    def init(self):
        self.minute_rsi = self.I(ta.rsi,pd.Series(self.data.Close),self.rsi_length_minute)
        self.minute5_rsi = resample_apply(self.re,ta.rsi,self.data.Close,self.rsi_length_2)
        
    
    def next(self):

        price = self.data.Close[-1]
        datetime = self.data.index[-1]
        if (datetime.time() > dt.time(hour=15,minute=20)) and self.position:
            self.position.close()

        if crossover(self.minute_rsi,self.upper_bound_minute) and self.minute5_rsi[-1] > self.upper_bound_2 :
            self.position.close()

        if crossover(self.minute_rsi,self.lower_bound_minute) and self.minute5_rsi[-1] > self.lower_bound_2 :
            self.buy(sl=0.995*price)


In [92]:
# Creating bt Engine to perform strategy on data
bt = Backtest(data,strategy1,cash=100000)

#running the engine 
stats = bt.run()

stats

Start                     2024-04-02 09:15...
End                       2024-04-10 15:29...
Duration                      8 days 06:14:00
Exposure Time [%]                   73.816794
Equity Final [$]                102297.169097
Equity Peak [$]                 102589.669097
Return [%]                           2.297169
Buy & Hold Return [%]                2.226388
Return (Ann.) [%]                  136.261715
Volatility (Ann.) [%]                43.18817
Sharpe Ratio                          3.15507
Sortino Ratio                       17.706956
Calmar Ratio                        66.946375
Max. Drawdown [%]                   -2.035386
Avg. Drawdown [%]                   -0.260687
Max. Drawdown Duration        4 days 23:14:00
Avg. Drawdown Duration        0 days 06:47:00
# Trades                                    9
Win Rate [%]                        44.444444
Best Trade [%]                       2.129404
Worst Trade [%]                     -0.511696
Avg. Trade [%]                    

In [93]:
bt.plot()

C:\Users\Ankit\AppData\Roaming\Python\Python310\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Ankit\AppData\Roaming\Python\Python310\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Ankit\AppData\Roaming\Python\Python310\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
C:\Users\Ankit\AppData\Roaming\Python\Python310\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p7623', ...)

In [96]:
stats = bt.optimize(
    #upper_bound_minute = range(55,90,1),
    #lower_bound_minute = range(20,50,1),
    #rsi_length_minute = range(8,16,1),
    upper_bound_2 = range(55,90,1),
    lower_bound_2 = range(20,50,1),
    #rsi_length_2 = range(4,18,1),
    #re = ['5T','10T','15T','30T'],
    maximize = 'Return [%]',
    #constraint= lambda param: param.upper_bound > param.lower_bound,
    #return_heatmap = True,
    max_tries = 1
)

stats



C:\Users\Ankit\AppData\Roaming\Python\Python310\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 1050 configurations.
  output = _optimize_grid()


Start                     2024-04-02 09:15...
End                       2024-04-10 15:29...
Duration                      8 days 06:14:00
Exposure Time [%]                   76.564885
Equity Final [$]                103706.658959
Equity Peak [$]                 103920.857507
Return [%]                           3.706659
Buy & Hold Return [%]                2.226388
Return (Ann.) [%]                  286.671663
Volatility (Ann.) [%]               56.762431
Sharpe Ratio                         5.050377
Sortino Ratio                       96.315252
Calmar Ratio                       219.374844
Max. Drawdown [%]                   -1.306766
Avg. Drawdown [%]                   -0.164382
Max. Drawdown Duration        2 days 22:13:00
Avg. Drawdown Duration        0 days 04:55:00
# Trades                                   17
Win Rate [%]                        52.941176
Best Trade [%]                       1.259334
Worst Trade [%]                     -0.511696
Avg. Trade [%]                    

In [97]:
bt.plot()

C:\Users\Ankit\AppData\Roaming\Python\Python310\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Ankit\AppData\Roaming\Python\Python310\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Ankit\AppData\Roaming\Python\Python310\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
C:\Users\Ankit\AppData\Roaming\Python\Python310\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p8051', ...)

In [80]:
plot_heatmaps(heatmap, agg='mean')

GridPlot(id='p7211', ...)

In [61]:
# Creating Strategy

class strategy2(Strategy):

    stoploss = 3
    trigger = 1

    sp = stoploss/(100*10)
    tr = trigger/(100*10)


    def init(self):
        #self.minute_rsi = self.I(ta.rsi,pd.Series(self.data.Close),self.rsi_length)
        #self.minute5_rsi = resample_apply('5T',ta.rsi,self.data.Close,self.rsi_length)
        pass
    
    def next(self):

        price = self.data.Close[-1]
        datetime = self.data.index[-1]
        
        if datetime.time() > dt.time(hour=15,minute=20):
            self.position.close()
        






        if not self.position and datetime.time() < dt.time(hour=15,minute=20):
            self.buy(tp= (1+self.tr)*price,sl=(1-self.sp)*price)
            

        


In [62]:
bt2 = Backtest(data,strategy2,cash=1000000)

#running the engine 
stats = bt2.run()

stats

True
15:21:00 15:20:00
day close
True
15:22:00 15:20:00
day close
True
15:23:00 15:20:00
day close
True
15:24:00 15:20:00
day close
True
15:25:00 15:20:00
day close
True
15:26:00 15:20:00
day close
True
15:27:00 15:20:00
day close
True
15:28:00 15:20:00
day close
True
15:29:00 15:20:00
day close
True
15:21:00 15:20:00
day close
True
15:22:00 15:20:00
day close
True
15:23:00 15:20:00
day close
True
15:24:00 15:20:00
day close
True
15:25:00 15:20:00
day close
True
15:26:00 15:20:00
day close
True
15:27:00 15:20:00
day close
True
15:28:00 15:20:00
day close
True
15:29:00 15:20:00
day close
True
15:21:00 15:20:00
day close
True
15:22:00 15:20:00
day close
True
15:23:00 15:20:00
day close
True
15:24:00 15:20:00
day close
True
15:25:00 15:20:00
day close
True
15:26:00 15:20:00
day close
True
15:27:00 15:20:00
day close
True
15:28:00 15:20:00
day close
True
15:29:00 15:20:00
day close
True
15:21:00 15:20:00
day close
True
15:22:00 15:20:00
day close
True
15:23:00 15:20:00
day close
True
15:24

Start                     2024-04-02 09:15...
End                       2024-04-10 15:29...
Duration                      8 days 06:14:00
Exposure Time [%]                    97.78626
Equity Final [$]               1020647.498289
Equity Peak [$]                1023249.897634
Return [%]                            2.06475
Buy & Hold Return [%]                2.226388
Return (Ann.) [%]                  124.511111
Volatility (Ann.) [%]                31.22281
Sharpe Ratio                         3.987825
Sortino Ratio                        27.98672
Calmar Ratio                        60.407163
Max. Drawdown [%]                   -2.061198
Avg. Drawdown [%]                   -0.265993
Max. Drawdown Duration        5 days 00:11:00
Avg. Drawdown Duration        0 days 07:20:00
# Trades                                  167
Win Rate [%]                        77.245509
Best Trade [%]                        0.20694
Worst Trade [%]                     -0.346379
Avg. Trade [%]                    

In [65]:
bt2.plot()

C:\Users\Ankit\AppData\Roaming\Python\Python310\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%d %b'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Ankit\AppData\Roaming\Python\Python310\site-packages\backtesting\_plotting.py:250: UserWarning: DatetimeFormatter scales now only accept a single format. Using the first provided: '%m/%Y'
  formatter=DatetimeTickFormatter(days=['%d %b', '%a %d'],
C:\Users\Ankit\AppData\Roaming\Python\Python310\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_drag' property; using the latest value
  fig = gridplot(
C:\Users\Ankit\AppData\Roaming\Python\Python310\site-packages\backtesting\_plotting.py:659: UserWarning: found multiple competing values for 'toolbar.active_scroll' property; using the latest value
  fig = gridplot(


GridPlot(id='p5628', ...)

In [64]:
stats,heatmap  = bt2.optimize(
    stoploss = range(1,10,1),
    trigger =  range(1,10,1),
    maximize = 'Return [%]',
    #constraint= lambda param: param.stoploss > param.trigger,
    return_heatmap = True,
    #max_tries = 100
)

heatmap

True
15:21:00 15:20:00
day close
True
15:22:00 15:20:00
day close
True
15:23:00 15:20:00
day close
True
15:24:00 15:20:00
day close
True
15:25:00 15:20:00
day close
True
15:26:00 15:20:00
day close
True
15:27:00 15:20:00
day close
True
15:28:00 15:20:00
day close
True
15:29:00 15:20:00
day close
True
15:21:00 15:20:00
day close
True
15:22:00 15:20:00
day close
True
15:23:00 15:20:00
day close
True
15:24:00 15:20:00
day close
True
15:25:00 15:20:00
day close
True
15:26:00 15:20:00
day close
True
15:27:00 15:20:00
day close
True
15:28:00 15:20:00
day close
True
15:29:00 15:20:00
day close
True
15:21:00 15:20:00
day close
True
15:22:00 15:20:00
day close
True
15:23:00 15:20:00
day close
True
15:24:00 15:20:00
day close
True
15:25:00 15:20:00
day close
True
15:26:00 15:20:00
day close
True
15:27:00 15:20:00
day close
True
15:28:00 15:20:00
day close
True
15:29:00 15:20:00
day close
True
15:21:00 15:20:00
day close
True
15:22:00 15:20:00
day close
True
15:23:00 15:20:00
day close
True
15:24

stoploss  trigger
1         1          2.06475
          2          2.06475
          3          2.06475
          4          2.06475
          5          2.06475
                      ...   
9         5          2.06475
          6          2.06475
          7          2.06475
          8          2.06475
          9          2.06475
Name: Return [%], Length: 81, dtype: float64